In [4]:
import pandas as pd
import numpy as np
from scipy.spatial import ConvexHull

import glob
import os

folder_path = 'combined_csvs'  # Change this to your actual folder path
pattern = '*.csv'

# Get list of all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, pattern))


df1 = pd.read_csv(csv_files[0]) 
df1

,POSITION X,POSITION Y,POSITION Z,UNIT,CATEGORY,COLLECTION,TIME,SET 1,ID,UNNAMED: 9,CLUSTER,CD68,LY6G
0,51257.254,42903.758,26,µm,Surface,Position,1,Class A,3,0,0,0,0
1,51619.285,42545.438,12,µm,Surface,Position,1,Class A,4,0,0,0,0
2,51489.516,42677.035,14,µm,Surface,Position,1,Class A,5,0,0,0,0
3,51618.191,42549.020,16,µm,Surface,Position,1,Class A,9,0,0,0,0
4,51434.113,42733.695,17,µm,Surface,Position,1,Class A,10,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115110,48725.254,40357.273,24,µm,Surface,Position,1,Class B,115202,0,0,0,0
115111,48741.340,40339.059,24,µm,Surface,Position,1,Class B,115203,0,0,0,0
115112,48718.805,40368.750,25,µm,Surface,Position,1,Class B,115206,0,0,0,0
115113,48733.887,40358.613,24,µm,Surface,Position,1,Class B,115207,0,0,0,0


In [5]:
# # Test case position data for dapi 

# dapi2_position = pd.read_csv('edge_test/tumor edge test DAPI position_Selection.csv' , skiprows=3)
# dapi2_position.drop(columns=['Unit', 'Category',
#        'Collection', 'Time', 'CC3 catchall', 'CC3 real', 'CD103',
#        'CD103+ cDC1', 'CD3','CD177','CD3-CD68', 'CD3-CD68+CD206', 'CD4',
#        'CD45',  'Unnamed: 18'], inplace=True)

# dapi2_distance = pd.read_csv('edge_test/tumor edge test_DAPI distance to tumor edge_Selection.csv' , skiprows=3)
# dapi2_distance.drop(columns=['Category', 'Surfaces', 'Time',
#        'CC3 catchall', 'CC3 real', 'CD103', 'CD103+ cDC1', 'CD177', 'CD3',
#        'CD3-CD68', 'CD3-CD68+CD206', 'Unit','CD4', 'CD45', 'Unnamed: 16'], inplace=True)

# # cd3_position.head()
# dapi2_final = dapi2_position.merge(dapi2_distance, on=["ID"], how='left').sort_values(by=['ID'], ascending=True)
# dapi2_final =  dapi2_final[['ID','Position X', 'Position Y', 'Position Z',
#        'Shortest Distance to Surfaces']].set_index('ID')
# dapi2_final.head()

In [ ]:

# 1) Load your data
# columns “x” and “y”
pts = df1[['Position X', 'Position Y']].values             # shape (N,2) array

# 2) Compute the convex hull
hull = ConvexHull(pts)

# hull.simplices is an (E,2) array of index pairs (i, j)
# each representing an edge between pts[i] and pts[j].
edges = hull.simplices

# 3) Define a vectorized distance‐to‐segment function
def point_to_segment_distances(points, seg_start, seg_end):
    """
    points: (N,2) array
    seg_start, seg_end: each (2,) array
    returns: (N,) array of distances from each point to the segment
    """
    v = seg_end - seg_start                      # (2,)
    w = points - seg_start                       # (N,2)
    # projection factor of each point onto the infinite line
    t = np.einsum('ij,j->i', w, v) / np.dot(v, v)
    # clamp to [0,1] so we “snap” to the segment
    t_clamped = np.clip(t, 0.0, 1.0)             # (N,)
    proj = seg_start + t_clamped[:,None] * v     # (N,2)
    # euclidean distance from each point to its projection
    return np.linalg.norm(points - proj, axis=1)

# 4) Loop over edges, keep the minimum distance per point
all_dists = np.full(len(pts), np.inf)
for i, j in edges:
    start, end = pts[i], pts[j]
    d = point_to_segment_distances(pts, start, end)
    all_dists = np.minimum(all_dists, d)

# 5) Attach back to your DataFrame
df['dist_to_hull'] = all_dists

# 6) (Optional) Inspect
df

KeyError: "None of [Index(['Position X', 'Position Y'], dtype='object')] are in the [columns]"